# Evolving a PDF

## Method 1: Using eko and ekomark.apply_pdf

In this first part we will compute the eko and subsequently apply the initial PDF "manually" calling a dedicated function. 

In [1]:
import eko

Again, we need to recompute the operator, since we didn't save it.

In [2]:
from banana.data.theories import default_card as th_card
from ekomark.data.operators import default_card as op_card

# here we replace the grid with a very minimal one, to speed up the example
op_card["interpolation_xgrid"] = [1e-3, 1e-2, 1e-1, 5e-1, 1.]
# let's compute two operators: one reaching up to Q2=100 GeV^2 and a second reaching up to Q2=10000 GeV^2
op_card["Q2grid"] = [100., 10000.]
evolution_operator = eko.run_dglap(th_card, op_card)

Next, we need to load the PDF that we want to evolve. EKO uses the same interface as lhapdf to query for the actual values of PDFs. However, for the scope of this tutorial we want to avoid the complication of dealing with an external dependency. Therefore we will use the toy PDFs as they were established by the Les Houches benchmark setting.

In [3]:
from banana import toy
pdf = toy.mkPDF("", 0)

Now, we have all ingredients at hand to evolve the PDF set with the operator:

In [4]:
from ekomark.apply import apply_pdf
evolved_pdfs = apply_pdf(evolution_operator, pdf)

The returned object `evolved_pdfs` is a dictionary which maps the requested final scales onto further dictionaries:

In [5]:
evolved_pdfs.keys()

dict_keys([100.0, 10000.0])

Each final scale contains a dictionary with a key `pdfs`, where all PDF values are hold, and a key `errors` , where the integration errors are hold.
Finally, inside each of those there is a mapping of Monte Carlo particle identifiers onto a the PDF value at the requested interpolation points.

E.g. to access the gluon PDF at `Q^2 = 100 GeV^2` you can run:

In [6]:
evolved_pdfs[100.0]["pdfs"][21]

array([ 2.17034797e+04,  3.40247532e+02,  6.64325189e+01, -2.09652559e+01,
        0.00000000e+00])

Note that we return the actual PDF and not the momentum fraction times the PDF.

## Method 2: Using ekobox

In this second part we illustrate how to get (and install) directly a LHAPDF set evolved with eko. 

First, we define our initial PDF. Here, we will use the same toy PDF as in the previous example, but any LHAPDF-like object will do the job!

In [7]:
from banana import toy
pdf = toy.mkPDF("", 0)

Now, we set the theory inputs: in this example we will evolve our toy PDF at LO and create a new LHAPDF object with
a size two `Q2grid`.

In [8]:
from banana.data.theories import default_card as th_card
from ekomark.data.operators import default_card as op_card

# here we replace the grid with a very minimal one, to speed up the example
op_card["interpolation_xgrid"] = [1e-3, 1e-2, 1e-1, 5e-1, 1.]
op_card["Q2grid"] = [100., 10000.]
# set QCD LO evolution
th_card["PTO"] = 0

Finally, we are ready to run eko and install the new PDF set.
Note, that if the evolved PDF already exist the code will raise an error and return.

Additionally, you can set `path` to load a precomputed EKO, while setting `store_path` you can save the produced EKO and reuse it later.
You can also iterate on the given PDF objects (e.g. replicas).

In [9]:
from ekobox.evol_pdf import evolve_pdfs
evolve_pdfs(
    [pdf],
    th_card,
    op_card,
    install=True,
    name="Evolved_PDF",
)

install_pdf Evolved_PDF


Now, you can access the evolved PDF as all the other PDF sets (note that this requires the Python interface of lhapdf).

In [10]:
import lhapdf
evolved_pdf = lhapdf.mkPDF("Evolved_PDF", 0)

LHAPDF 6.4.0 loading /home/felix/local/share/LHAPDF/Evolved_PDF/Evolved_PDF_0000.dat
Evolved_PDF PDF set, member #0, version 1


To obtain the value of the gluon PDF at a given scale you can simply do:

In [11]:
pid = 21 # gluon pid
Q2 = 89.10 #  Q^2 in Gev^2
x = 0.01 # momentum fraction 

# check that the particle is present
print("has gluon?",evolved_pdf.hasFlavor(pid))
# now do the lookup
xg = evolved_pdf.xfxQ2(pid, x, Q2)
print(f"xg(x={x}, Q2={Q2}) = {xg}")

has gluon? True
xg(x=0.01, Q2=89.1) = 3.4255366354811705


## A more Realistic Example: Benchmark to CT14llo

In this part of the tutorial we do an eko benchmark showing how PDFs evolved with eko can reproduce the values from the original LHAPDF grids.

First, we need to set up the theory and operator runcards to match the settings used to produce the chosen PDF, here we will use `CT14llo`.

We have to use LO evolution and we choose to dump our PDF into grids with 5 values of `Q2` and 60 points in x-space logarithmically spaced between 1e-7 and 0.1 and linearly spaced from 0.1 to 1. 

In [12]:
import numpy as np
import lhapdf
from banana.data.theories import default_card as th_card
from ekomark.data.operators import default_card as op_card
from eko.interpolation import make_grid

# get the PDF object
ct14llo = lhapdf.mkPDF("CT14llo")

# setup the operator card
op_card["interpolation_xgrid"] = make_grid(30, 30).tolist() # x grid
op_card["Q2grid"] = np.geomspace(10., 1e5, 5).tolist() # Q2 grid

# setup the theory card - this can be mostly inferred from the PDF's .info file
th_card["PTO"] = 0 # QCD LO
th_card["mc"] = 1.3 # charm mass
th_card["mb"] = 4.75 # bottom mass
th_card["mt"] = 172. # top mass
th_card["alphas"] = 0.130000 # reference value of alpha_s
th_card["Qref"] = 91.1876 # the reference scale at which alpha_s is provided
th_card["nfref"] = 5 # the number of flavors active at the alpha_s reference scale
th_card["MaxNfAs"]= 5  # the maximum number of flavors active in the alpha_s evolution
th_card["Q0"] = 1.295000 # starting point for the evolution 
th_card["nf0"] = 3 # the number of flavors active at the reference scale
th_card["MaxNfPdf"] = 5  # the maximum number of flavors active in the pdf evolution

LHAPDF 6.4.0 loading /home/felix/local/share/LHAPDF/CT14llo/CT14llo_0000.dat
CT14llo PDF set, member #0, version 1; LHAPDF ID = 13205


Next, we run the evolution using method 2 and save the new PDF. Due to the extended x grid and Q2 grid this might take a minute so please be patient ...

In [13]:
from ekobox.evol_pdf import evolve_pdfs

evolve_pdfs(
    [ct14llo],
    th_card,
    op_card,
    install=True,
    name="my_ct14llo",
)

install_pdf my_ct14llo


Now, we can compare the values given by the original PDF set and the one evolved with eko, both at different `x` and `Q2` scales, for a chosen parton,
here we look at the gluon:

In [14]:
import pandas as pd

# load evolved pdf
my_ct14llo = lhapdf.mkPDF("my_ct14llo", 0)

pid = 21 # gluon pid

# collect data
log = {"x": [], "Q2" : [],  "ct14llo": [], "my_ct14llo": [], "relative_diff": []} 
for q2 in np.geomspace(10., 1e5, 5):
    for x in np.geomspace(1e-5, 0.9, 5):
        value = ct14llo.xfxQ2(pid, x, q2)
        my_value =  my_ct14llo.xfxQ2(pid, x, q2)
        log["x"].append(x)
        log["Q2"].append(q2)
        log["ct14llo"].append(value)
        log["my_ct14llo"].append(my_value)
        log["relative_diff"].append((value - my_value) / value)

print(pd.DataFrame(log))

LHAPDF 6.4.0 loading /home/felix/local/share/LHAPDF/my_ct14llo/my_ct14llo_0000.dat
my_ct14llo PDF set, member #0, version 1
           x        Q2       ct14llo    my_ct14llo  relative_diff
0   0.000010      10.0  4.560628e+01  4.559560e+01       0.000234
1   0.000173      10.0  2.185377e+01  2.184884e+01       0.000225
2   0.003000      10.0  8.734759e+00  8.732856e+00       0.000218
3   0.051962      10.0  1.873738e+00  1.874169e+00      -0.000230
4   0.900000      10.0  4.278472e-05  4.346894e-05      -0.015992
5   0.000010     100.0  1.288848e+02  1.287965e+02       0.000685
6   0.000173     100.0  4.661146e+01  4.657867e+01       0.000703
7   0.003000     100.0  1.323922e+01  1.322967e+01       0.000721
8   0.051962     100.0  1.978974e+00  1.979055e+00      -0.000041
9   0.900000     100.0  7.121784e-06  7.245044e-06      -0.017308
10  0.000010    1000.0  2.272552e+02  2.270758e+02       0.000790
11  0.000173    1000.0  6.959939e+01  6.954205e+01       0.000824
12  0.003000    10

As you can see EKO is able to reproduce the numbers from the original LHAPDF grid mostly below the permille level.

The accuracy is mainly limited by the number of points in the `x` and `Q2` grids that can be finer to achieve higher precision.

You can also notice that at large-x the gluon pdf vanishes so the worst accuracy of our benchmark is not worrying at all. 